## 📦 Step 1: Install Required Libraries

First, install all dependencies. This takes ~2-3 minutes.

In [ ]:
!pip install -q sentence-transformers
!pip install -q faiss-cpu
!pip install -q beir
!pip install -q pandas matplotlib seaborn

print("✅ All libraries installed!")

## 📥 Step 2: Import Libraries and Download Dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm.auto import tqdm

# Sentence Transformers for embeddings
from sentence_transformers import SentenceTransformer

# FAISS for indexes
import faiss

# BEIR for dataset
from beir import util
from beir.datasets.data_loader import GenericDataLoader

# Set style for plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ Libraries imported successfully!")

In [ ]:
# Download SciFact dataset (smallest BEIR dataset - only 5K docs)
dataset_name = "scifact"
url = f"https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{dataset_name}.zip"

print(f"Downloading {dataset_name} dataset...")
data_path = util.download_and_unzip(url, "datasets")

# Load dataset
print("Loading dataset...")
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

print(f"\n✅ Dataset loaded!")
print(f"   Documents: {len(corpus):,}")
print(f"   Queries: {len(queries):,}")
print(f"   Relevance judgments: {len(qrels):,}")

## 🔍 Step 3: Explore the Dataset

In [ ]:
# Show example document
doc_id = list(corpus.keys())[0]
print("Example Document:")
print(f"ID: {doc_id}")
print(f"Title: {corpus[doc_id]['title']}")
print(f"Text (first 200 chars): {corpus[doc_id]['text'][:200]}...")

print("\n" + "="*80 + "\n")

# Show example query
query_id = list(queries.keys())[0]
print("Example Query:")
print(f"ID: {query_id}")
print(f"Text: {queries[query_id]}")

print("\n" + "="*80 + "\n")

# Show example relevance judgment
print("Example Relevance Judgment:")
print(f"Query {query_id} relevant documents: {qrels[query_id]}")

## 🧠 Step 4: Load Embedding Model and Encode Documents

We use **BGE-base-en-v1.5** - the same model used in the paper.
This produces 768-dimensional vectors.

### ⚡ IMPORTANT: Speed Up Encoding by 10-20x!

**If encoding is too slow (>3 minutes), enable GPU:**

1. Click: **Runtime** → **Change runtime type**
2. Select: **Hardware accelerator** → **T4 GPU**  
3. Click: **Save**
4. Re-run from the beginning

**Encoding speed comparison:**
- CPU: ~3-5 minutes for SciFact (5K docs)
- GPU (T4): ~20-30 seconds ⚡

The optimizations below (batch_size=128) already make it 4x faster, but GPU gives another 10x boost!

In [ ]:
# Load model (same as paper)
model_name = 'BAAI/bge-base-en-v1.5'
print(f"Loading model: {model_name}")
print("This may take 1-2 minutes on first run...")

model = SentenceTransformer(model_name)
dimension = model.get_sentence_embedding_dimension()

print(f"\n✅ Model loaded!")
print(f"   Embedding dimension: {dimension}")

In [ ]:
# Prepare documents for encoding
doc_ids = list(corpus.keys())
doc_texts = [corpus[did]['title'] + ' ' + corpus[did]['text'] for did in doc_ids]

print(f"Encoding {len(doc_texts):,} documents...")
print("This takes ~2-3 minutes for SciFact...")

# Encode all documents
doc_embeddings = model.encode(
    doc_texts,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True  # Important for cosine similarity!
)

print(f"\n✅ Documents encoded!")
print(f"   Shape: {doc_embeddings.shape}")
print(f"   Memory: {doc_embeddings.nbytes / (1024**2):.2f} MB")

In [ ]:
# Encode queries
query_ids = list(queries.keys())
query_texts = [queries[qid] for qid in query_ids]

print(f"Encoding {len(query_texts):,} queries...")

query_embeddings = model.encode(
    query_texts,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print(f"\n✅ Queries encoded!")
print(f"   Shape: {query_embeddings.shape}")

## 🗂️ Step 5: Build Indexes

### 5.1: Flat Index (Baseline - Exact Search)

In [ ]:
print("Building Flat Index (exact search)...")
start_time = time.time()

# Create flat index (uses inner product for cosine similarity after normalization)
flat_index = faiss.IndexFlatIP(dimension)
flat_index.add(doc_embeddings.astype('float32'))

build_time_flat = time.time() - start_time

print(f"\n✅ Flat Index built!")
print(f"   Build time: {build_time_flat:.3f} seconds")
print(f"   Number of vectors: {flat_index.ntotal:,}")
print(f"   Index type: Exact search (100% recall)")

### 5.2: HNSW Index (Approximate Search - Fast)

In [ ]:
print("Building HNSW Index (approximate search)...")
start_time = time.time()

# HNSW parameters (from paper)
M = 16                  # Number of connections per node
ef_construction = 100   # Build quality
ef_search = 50          # Search quality (adjustable)

# Create HNSW index
hnsw_index = faiss.IndexHNSWFlat(dimension, M)
hnsw_index.hnsw.efConstruction = ef_construction
hnsw_index.add(doc_embeddings.astype('float32'))

# Set search parameter
hnsw_index.hnsw.efSearch = ef_search

build_time_hnsw = time.time() - start_time

print(f"\n✅ HNSW Index built!")
print(f"   Build time: {build_time_hnsw:.3f} seconds")
print(f"   Number of vectors: {hnsw_index.ntotal:,}")
print(f"   Parameters: M={M}, efConstruction={ef_construction}, efSearch={ef_search}")
print(f"   Build time vs Flat: {build_time_hnsw/build_time_flat:.2f}x")

### 5.3: HNSW with INT8 Quantization (Memory Efficient)

In [ ]:
print("Building HNSW Index with INT8 quantization...")
start_time = time.time()

# Create quantized HNSW index (4x memory reduction!)
hnsw_int8_index = faiss.IndexHNSWSQ(dimension, faiss.ScalarQuantizer.QT_8bit, M)
hnsw_int8_index.hnsw.efConstruction = ef_construction

# IMPORTANT: Train the quantizer first!
print("   Training quantizer on document embeddings...")
hnsw_int8_index.train(doc_embeddings.astype('float32'))

# Now add the vectors
print("   Adding vectors to index...")
hnsw_int8_index.add(doc_embeddings.astype('float32'))

# Set search parameter
hnsw_int8_index.hnsw.efSearch = ef_search

build_time_hnsw_int8 = time.time() - start_time

print(f"\n✅ HNSW INT8 Index built!")
print(f"   Build time: {build_time_hnsw_int8:.3f} seconds")
print(f"   Parameters: M={M}, efConstruction={ef_construction}, efSearch={ef_search}, Quantization=INT8")

## 🔬 Step 6: Run Search Experiments

Now we search with all three indexes and measure performance.

In [ ]:
def search_and_measure(index, query_embeddings, k=10, name="Index"):
    """
    Search with an index and measure latency.
    """
    print(f"\nSearching with {name}...")
    
    latencies = []
    all_indices = []
    all_scores = []
    
    # Search each query individually to measure latency
    for i, query_emb in enumerate(tqdm(query_embeddings, desc=f"{name} search")):
        start = time.time()
        scores, indices = index.search(query_emb.reshape(1, -1).astype('float32'), k)
        latency = (time.time() - start) * 1000  # Convert to ms
        
        latencies.append(latency)
        all_indices.append(indices[0])
        all_scores.append(scores[0])
    
    latencies = np.array(latencies)
    
    print(f"\n✅ {name} Search Complete!")
    print(f"   Total queries: {len(latencies):,}")
    print(f"   Latency (median): {np.median(latencies):.3f} ms")
    print(f"   Latency (p95): {np.percentile(latencies, 95):.3f} ms")
    print(f"   Latency (p99): {np.percentile(latencies, 99):.3f} ms")
    print(f"   QPS (approx): {1000 / np.median(latencies):.1f} queries/second")
    
    return {
        'name': name,
        'indices': np.array(all_indices),
        'scores': np.array(all_scores),
        'latencies': latencies,
        'median_latency': np.median(latencies),
        'p95_latency': np.percentile(latencies, 95),
        'p99_latency': np.percentile(latencies, 99),
    }

In [ ]:
# Search with all indexes
k = 10  # Retrieve top-10 results

results_flat = search_and_measure(flat_index, query_embeddings, k=k, name="Flat")
results_hnsw = search_and_measure(hnsw_index, query_embeddings, k=k, name="HNSW")
results_hnsw_int8 = search_and_measure(hnsw_int8_index, query_embeddings, k=k, name="HNSW-INT8")

## 📊 Step 7: Evaluate Retrieval Quality

Calculate Recall@10 and nDCG@10 for each index.

In [ ]:
def calculate_recall(retrieved_indices, qrels, query_ids, doc_ids, k=10):
    """
    Calculate Recall@k: fraction of relevant docs found in top-k.
    """
    recalls = []
    
    for i, qid in enumerate(query_ids):
        if qid not in qrels:
            continue
        
        # Get relevant documents for this query
        relevant_docs = set(qrels[qid].keys())
        
        # Get retrieved documents (convert indices to doc_ids)
        retrieved_docs = set([doc_ids[idx] for idx in retrieved_indices[i][:k] if idx >= 0])
        
        # Calculate recall
        if len(relevant_docs) > 0:
            recall = len(relevant_docs & retrieved_docs) / len(relevant_docs)
            recalls.append(recall)
    
    return np.mean(recalls)


def calculate_ndcg(retrieved_indices, qrels, query_ids, doc_ids, k=10):
    """
    Calculate nDCG@k: normalized discounted cumulative gain.
    """
    ndcgs = []
    
    for i, qid in enumerate(query_ids):
        if qid not in qrels:
            continue
        
        relevant_docs = qrels[qid]
        retrieved_docs = [doc_ids[idx] for idx in retrieved_indices[i][:k] if idx >= 0]
        
        # Calculate DCG
        dcg = 0
        for rank, doc_id in enumerate(retrieved_docs, 1):
            relevance = relevant_docs.get(doc_id, 0)
            dcg += (2 ** relevance - 1) / np.log2(rank + 1)
        
        # Calculate ideal DCG (perfect ranking)
        ideal_relevances = sorted(relevant_docs.values(), reverse=True)[:k]
        idcg = sum((2 ** rel - 1) / np.log2(rank + 2) for rank, rel in enumerate(ideal_relevances))
        
        # Calculate nDCG
        if idcg > 0:
            ndcg = dcg / idcg
            ndcgs.append(ndcg)
    
    return np.mean(ndcgs)

In [ ]:
# Calculate metrics for all indexes
print("Calculating quality metrics...\n")

for results in [results_flat, results_hnsw, results_hnsw_int8]:
    recall = calculate_recall(results['indices'], qrels, query_ids, doc_ids, k=10)
    ndcg = calculate_ndcg(results['indices'], qrels, query_ids, doc_ids, k=10)
    
    results['recall@10'] = recall
    results['ndcg@10'] = ndcg
    
    print(f"{results['name']}:")
    print(f"   Recall@10: {recall:.4f}")
    print(f"   nDCG@10: {ndcg:.4f}")
    print()

## 📈 Step 8: Visualize Results

Create comparison charts to understand the trade-offs.

In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame([
    {
        'Index': results['name'],
        'Recall@10': results['recall@10'],
        'nDCG@10': results['ndcg@10'],
        'Median Latency (ms)': results['median_latency'],
        'P95 Latency (ms)': results['p95_latency'],
        'P99 Latency (ms)': results['p99_latency'],
    }
    for results in [results_flat, results_hnsw, results_hnsw_int8]
])

print("\n📊 COMPARISON TABLE")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

In [ ]:
# Plot 1: Speed vs Quality Trade-off
fig, ax = plt.subplots(figsize=(10, 6))

for results in [results_flat, results_hnsw, results_hnsw_int8]:
    ax.scatter(
        results['median_latency'],
        results['ndcg@10'],
        s=200,
        alpha=0.7,
        label=results['name']
    )
    ax.annotate(
        results['name'],
        (results['median_latency'], results['ndcg@10']),
        xytext=(10, 10),
        textcoords='offset points',
        fontsize=10,
        bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.3)
    )

ax.set_xlabel('Median Latency (ms)', fontsize=12)
ax.set_ylabel('nDCG@10', fontsize=12)
ax.set_title('Speed vs Quality Trade-off (SciFact Dataset)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=10)

plt.tight_layout()
plt.savefig('speed_vs_quality.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Plot saved as 'speed_vs_quality.png'")

In [ ]:
# Plot 2: Latency Distribution Comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 4), sharey=True)

for ax, results in zip(axes, [results_flat, results_hnsw, results_hnsw_int8]):
    ax.hist(results['latencies'], bins=30, alpha=0.7, edgecolor='black')
    ax.axvline(results['median_latency'], color='red', linestyle='--', linewidth=2, label='Median')
    ax.axvline(results['p95_latency'], color='orange', linestyle='--', linewidth=2, label='P95')
    ax.set_xlabel('Latency (ms)', fontsize=10)
    ax.set_title(f"{results['name']}\n(median: {results['median_latency']:.2f}ms)", fontsize=11, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

axes[0].set_ylabel('Frequency', fontsize=10)
fig.suptitle('Query Latency Distributions', fontsize=14, fontweight='bold', y=1.02)

plt.tight_layout()
plt.savefig('latency_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Plot saved as 'latency_distributions.png'")

In [ ]:
# Plot 3: Bar Chart Comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Quality metrics
x = np.arange(len(comparison_df))
width = 0.35

axes[0].bar(x - width/2, comparison_df['Recall@10'], width, label='Recall@10', alpha=0.8)
axes[0].bar(x + width/2, comparison_df['nDCG@10'], width, label='nDCG@10', alpha=0.8)
axes[0].set_ylabel('Score', fontsize=11)
axes[0].set_title('Retrieval Quality Comparison', fontsize=12, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(comparison_df['Index'])
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_ylim([0, 1.0])

# Latency metrics
axes[1].bar(x, comparison_df['Median Latency (ms)'], alpha=0.8, color='steelblue')
axes[1].set_ylabel('Latency (ms)', fontsize=11)
axes[1].set_title('Query Latency Comparison (Median)', fontsize=12, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(comparison_df['Index'])
axes[1].grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for ax in axes:
    for container in ax.containers:
        ax.bar_label(container, fmt='%.3f', fontsize=9)

plt.tight_layout()
plt.savefig('comparison_bars.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Plot saved as 'comparison_bars.png'")

## 📝 Step 9: Key Findings Summary

In [ ]:
# Calculate speedups and quality preservation
speedup_hnsw = results_flat['median_latency'] / results_hnsw['median_latency']
speedup_hnsw_int8 = results_flat['median_latency'] / results_hnsw_int8['median_latency']

quality_loss_hnsw = (1 - results_hnsw['ndcg@10'] / results_flat['ndcg@10']) * 100
quality_loss_hnsw_int8 = (1 - results_hnsw_int8['ndcg@10'] / results_flat['ndcg@10']) * 100

print("\n" + "="*80)
print("🎯 KEY FINDINGS")
print("="*80)

print(f"\n1. SPEED:")
print(f"   • HNSW is {speedup_hnsw:.2f}x faster than Flat index")
print(f"   • HNSW-INT8 is {speedup_hnsw_int8:.2f}x faster than Flat index")

print(f"\n2. QUALITY:")
print(f"   • HNSW preserves {100-quality_loss_hnsw:.2f}% of Flat quality (loss: {quality_loss_hnsw:.2f}%)")
print(f"   • HNSW-INT8 preserves {100-quality_loss_hnsw_int8:.2f}% of Flat quality (loss: {quality_loss_hnsw_int8:.2f}%)")

print(f"\n3. BUILD TIME:")
print(f"   • Flat: {build_time_flat:.3f}s (fastest to build)")
print(f"   • HNSW: {build_time_hnsw:.3f}s ({build_time_hnsw/build_time_flat:.2f}x slower)")
print(f"   • HNSW-INT8: {build_time_hnsw_int8:.3f}s ({build_time_hnsw_int8/build_time_flat:.2f}x slower)")

print(f"\n4. MEMORY:")
print(f"   • Documents: {doc_embeddings.nbytes / (1024**2):.2f} MB (float32)")
print(f"   • INT8 would use ~{doc_embeddings.nbytes / (1024**2) / 4:.2f} MB (4x compression)")

print(f"\n5. RECOMMENDATION for SciFact (5K docs):")
if speedup_hnsw < 2:
    print(f"   ➡️ Flat index is sufficient (speedup not significant)")
else:
    print(f"   ➡️ HNSW recommended (significant speedup with minimal quality loss)")

print("\n" + "="*80)

## 💾 Step 10: Save Results

In [ ]:
# Save comparison table
comparison_df.to_csv('experiment_results.csv', index=False)
print("✅ Results saved to 'experiment_results.csv'")

# Save detailed latency data
latency_df = pd.DataFrame({
    'Flat': results_flat['latencies'],
    'HNSW': results_hnsw['latencies'],
    'HNSW-INT8': results_hnsw_int8['latencies']
})
latency_df.to_csv('latency_data.csv', index=False)
print("✅ Latency data saved to 'latency_data.csv'")

## 🚀 Next Steps

Congratulations! You've completed your first experiments. Here's what you can do next:

### Option 1: Test Different Parameters
- Try different `efSearch` values: [10, 30, 50, 100, 200]
- See how it affects speed vs quality trade-off

### Option 2: Test on Larger Dataset
- Try FiQA (57K docs) or NQ (2.7M docs)
- See how speedup increases with dataset size

### Option 3: Add BM25 Baseline
- Compare dense (neural) vs sparse (BM25) retrieval
- Implement hybrid search (combine both)

### Option 4: Memory Constraints Study
- Test different quantization levels
- Measure quality vs memory trade-offs

---

**Great job!** 🎉 You've successfully replicated the core experiments from the paper!

## 🔧 BONUS: Parameter Tuning - Testing Different efSearch Values

Let's explore how the `efSearch` parameter affects the speed vs quality trade-off for HNSW.

In [ ]:
# Test different efSearch values
efSearch_values = [10, 20, 30, 50, 75, 100, 150, 200]

print("🔬 Testing different efSearch parameters...")
print(f"Values to test: {efSearch_values}")
print(f"This will take ~{len(efSearch_values)} minutes...\n")

results_by_efSearch = []

for ef in tqdm(efSearch_values, desc="Testing efSearch values"):
    # Set the efSearch parameter
    hnsw_index.hnsw.efSearch = ef
    
    # Run searches and measure performance
    latencies = []
    all_indices = []
    
    for query_emb in query_embeddings:
        start = time.time()
        scores, indices = hnsw_index.search(query_emb.reshape(1, -1).astype('float32'), k=10)
        latency = (time.time() - start) * 1000  # Convert to milliseconds
        latencies.append(latency)
        all_indices.append(indices[0])
    
    latencies = np.array(latencies)
    all_indices = np.array(all_indices)
    
    # Calculate quality metrics
    recall = calculate_recall(all_indices, qrels, query_ids, doc_ids, k=10)
    ndcg = calculate_ndcg(all_indices, qrels, query_ids, doc_ids, k=10)
    
    # Store results for this efSearch value
    results_by_efSearch.append({
        'efSearch': ef,
        'median_latency': np.median(latencies),
        'p95_latency': np.percentile(latencies, 95),
        'recall@10': recall,
        'ndcg@10': ndcg,
        'qps': 1000 / np.median(latencies)  # Queries per second
    })
    
    print(f"efSearch={ef:3d}: Latency={np.median(latencies):6.2f}ms, nDCG@10={ndcg:.4f}, Recall@10={recall:.4f}")

# Convert to DataFrame for easy analysis
efSearch_df = pd.DataFrame(results_by_efSearch)

print("\n✅ Parameter tuning complete!")
print("\n📊 RESULTS TABLE:")
print("="*90)
print(efSearch_df.to_string(index=False))
print("="*90)

In [ ]:
# Plot 1: Speed vs Quality Trade-off for different efSearch values
fig, ax = plt.subplots(figsize=(12, 7))

# Plot the curve
ax.plot(efSearch_df['median_latency'], efSearch_df['ndcg@10'], 
        marker='o', markersize=8, linewidth=2, color='steelblue', label='HNSW')

# Annotate each point with efSearch value
for idx, row in efSearch_df.iterrows():
    ax.annotate(f"ef={row['efSearch']}", 
                (row['median_latency'], row['ndcg@10']),
                xytext=(5, 5), textcoords='offset points',
                fontsize=9, alpha=0.8)

# Add reference line for Flat index
ax.axhline(y=results_flat['ndcg@10'], color='red', linestyle='--', 
           linewidth=2, label=f"Flat Index (nDCG={results_flat['ndcg@10']:.4f})", alpha=0.7)

ax.set_xlabel('Median Latency (ms)', fontsize=12, fontweight='bold')
ax.set_ylabel('nDCG@10', fontsize=12, fontweight='bold')
ax.set_title('HNSW: Speed vs Quality Trade-off for Different efSearch Values', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=11, loc='lower right')

plt.tight_layout()
plt.savefig('efSearch_speed_vs_quality.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Plot saved as 'efSearch_speed_vs_quality.png'")

In [ ]:
# Plot 2: Multi-metric comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 2a: Latency vs efSearch
axes[0, 0].plot(efSearch_df['efSearch'], efSearch_df['median_latency'], 
                marker='o', linewidth=2, color='steelblue', markersize=8)
axes[0, 0].fill_between(efSearch_df['efSearch'], 
                         efSearch_df['median_latency'], 
                         efSearch_df['p95_latency'], 
                         alpha=0.3, label='p50-p95 range')
axes[0, 0].set_xlabel('efSearch', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('Latency (ms)', fontsize=11, fontweight='bold')
axes[0, 0].set_title('Query Latency vs efSearch', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

# Plot 2b: nDCG vs efSearch
axes[0, 1].plot(efSearch_df['efSearch'], efSearch_df['ndcg@10'], 
                marker='s', linewidth=2, color='green', markersize=8)
axes[0, 1].axhline(y=results_flat['ndcg@10'], color='red', linestyle='--', 
                   linewidth=2, label='Flat Index', alpha=0.7)
axes[0, 1].set_xlabel('efSearch', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('nDCG@10', fontsize=11, fontweight='bold')
axes[0, 1].set_title('Retrieval Quality vs efSearch', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

# Plot 2c: Recall vs efSearch
axes[1, 0].plot(efSearch_df['efSearch'], efSearch_df['recall@10'], 
                marker='^', linewidth=2, color='orange', markersize=8)
axes[1, 0].axhline(y=results_flat['recall@10'], color='red', linestyle='--', 
                   linewidth=2, label='Flat Index', alpha=0.7)
axes[1, 0].set_xlabel('efSearch', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Recall@10', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Recall vs efSearch', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()

# Plot 2d: QPS vs efSearch
axes[1, 1].plot(efSearch_df['efSearch'], efSearch_df['qps'], 
                marker='D', linewidth=2, color='purple', markersize=8)
axes[1, 1].set_xlabel('efSearch', fontsize=11, fontweight='bold')
axes[1, 1].set_ylabel('Queries Per Second (QPS)', fontsize=11, fontweight='bold')
axes[1, 1].set_title('Throughput vs efSearch', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

fig.suptitle('HNSW Parameter Analysis: Impact of efSearch', 
             fontsize=15, fontweight='bold', y=0.995)

plt.tight_layout()
plt.savefig('efSearch_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Plot saved as 'efSearch_analysis.png'")

In [ ]:
# Find the optimal efSearch value (best quality/speed balance)
# Calculate efficiency score: quality / latency (higher is better)
efSearch_df['efficiency_score'] = efSearch_df['ndcg@10'] / (efSearch_df['median_latency'] / 1000)

optimal_idx = efSearch_df['efficiency_score'].idxmax()
optimal_efSearch = efSearch_df.loc[optimal_idx]

print("\n" + "="*90)
print("🎯 OPTIMAL efSearch VALUE")
print("="*90)
print(f"\nBest efficiency score at efSearch = {optimal_efSearch['efSearch']}")
print(f"\nMetrics:")
print(f"   • Median Latency: {optimal_efSearch['median_latency']:.2f} ms")
print(f"   • P95 Latency: {optimal_efSearch['p95_latency']:.2f} ms")
print(f"   • nDCG@10: {optimal_efSearch['ndcg@10']:.4f}")
print(f"   • Recall@10: {optimal_efSearch['recall@10']:.4f}")
print(f"   • QPS: {optimal_efSearch['qps']:.1f} queries/second")
print(f"   • Efficiency Score: {optimal_efSearch['efficiency_score']:.2f}")

print(f"\n📊 Comparison with extremes:")
print(f"\nLowest efSearch ({efSearch_df.iloc[0]['efSearch']}):")
print(f"   • {efSearch_df.iloc[0]['median_latency']:.2f}ms (fastest)")
print(f"   • nDCG@10: {efSearch_df.iloc[0]['ndcg@10']:.4f} ({(1-efSearch_df.iloc[0]['ndcg@10']/results_flat['ndcg@10'])*100:.1f}% quality loss vs Flat)")

print(f"\nHighest efSearch ({efSearch_df.iloc[-1]['efSearch']}):")
print(f"   • {efSearch_df.iloc[-1]['median_latency']:.2f}ms (slowest)")
print(f"   • nDCG@10: {efSearch_df.iloc[-1]['ndcg@10']:.4f} ({(1-efSearch_df.iloc[-1]['ndcg@10']/results_flat['ndcg@10'])*100:.1f}% quality loss vs Flat)")

print(f"\nOptimal efSearch ({optimal_efSearch['efSearch']}):")
print(f"   • {optimal_efSearch['median_latency']:.2f}ms (balanced)")
print(f"   • nDCG@10: {optimal_efSearch['ndcg@10']:.4f} ({(1-optimal_efSearch['ndcg@10']/results_flat['ndcg@10'])*100:.1f}% quality loss vs Flat)")

print(f"\n💡 RECOMMENDATION:")
print(f"   Use efSearch = {optimal_efSearch['efSearch']} for best quality/speed balance")
print(f"   This gives {optimal_efSearch['ndcg@10']/results_flat['ndcg@10']*100:.1f}% of Flat quality")
print(f"   at {results_flat['median_latency']/optimal_efSearch['median_latency']:.1f}x speedup!")
print("\n" + "="*90)

In [ ]:
# Save efSearch tuning results
efSearch_df.to_csv('efSearch_tuning_results.csv', index=False)
print("✅ Results saved to 'efSearch_tuning_results.csv'")

## 🔧 BONUS 2: M Parameter Tuning - Testing Different Graph Connectivity

Now let's explore how the `M` parameter affects HNSW performance. Unlike `efSearch`, changing `M` requires rebuilding the entire index for each value.

**What is M?**
- M = number of bidirectional connections per node in the graph
- Higher M = better quality but more memory and slower builds
- Typical range: 8-64 (paper uses M=16 as baseline)

**Trade-offs:**
- **Low M (8-12)**: Faster builds, less memory, lower quality
- **Medium M (16-24)**: Balanced (recommended)
- **High M (32-64)**: Best quality, but expensive builds and memory

In [ ]:
# Test different M values (graph connectivity)
M_values = [8, 12, 16, 24, 32]

print("🔬 Testing different M parameters...")
print(f"Values to test: {M_values}")
print(f"⚠️  WARNING: This requires rebuilding the index for each M value!")
print(f"This will take ~{len(M_values) * 2} minutes (longer than efSearch tuning)...\n")

results_by_M = []

for M_test in tqdm(M_values, desc="Testing M values"):
    print(f"\n{'='*70}")
    print(f"Building HNSW index with M={M_test}...")
    print(f"{'='*70}")
    
    # Build a new index with this M value
    start_build = time.time()
    index_M = faiss.IndexHNSWFlat(dimension, M_test)
    index_M.hnsw.efConstruction = ef_construction  # Use same efConstruction as baseline
    index_M.add(doc_embeddings.astype('float32'))
    index_M.hnsw.efSearch = ef_search  # Use same efSearch as baseline
    build_time = time.time() - start_build
    
    # Run searches and measure performance
    latencies = []
    all_indices = []
    
    for query_emb in query_embeddings:
        start = time.time()
        scores, indices = index_M.search(query_emb.reshape(1, -1).astype('float32'), k=10)
        latency = (time.time() - start) * 1000  # Convert to milliseconds
        latencies.append(latency)
        all_indices.append(indices[0])
    
    latencies = np.array(latencies)
    all_indices = np.array(all_indices)
    
    # Calculate quality metrics
    recall = calculate_recall(all_indices, qrels, query_ids, doc_ids, k=10)
    ndcg = calculate_ndcg(all_indices, qrels, query_ids, doc_ids, k=10)
    
    # Store results for this M value
    results_by_M.append({
        'M': M_test,
        'build_time': build_time,
        'median_latency': np.median(latencies),
        'p95_latency': np.percentile(latencies, 95),
        'recall@10': recall,
        'ndcg@10': ndcg,
        'qps': 1000 / np.median(latencies)  # Queries per second
    })
    
    print(f"\n✅ M={M_test} complete:")
    print(f"   Build time: {build_time:.3f}s")
    print(f"   Latency: {np.median(latencies):.2f}ms (median), {np.percentile(latencies, 95):.2f}ms (p95)")
    print(f"   nDCG@10: {ndcg:.4f}, Recall@10: {recall:.4f}")

# Convert to DataFrame for easy analysis
M_df = pd.DataFrame(results_by_M)

print("\n" + "="*90)
print("✅ M Parameter tuning complete!")
print("="*90)
print("\n📊 RESULTS TABLE:")
print("="*90)
print(M_df.to_string(index=False))
print("="*90)

In [ ]:
# Plot 1: M Parameter Impact - Speed vs Quality
fig, ax = plt.subplots(figsize=(12, 7))

# Plot the curve
ax.plot(M_df['median_latency'], M_df['ndcg@10'], 
        marker='o', markersize=10, linewidth=2, color='darkgreen', label='HNSW (varying M)')

# Annotate each point with M value
for idx, row in M_df.iterrows():
    ax.annotate(f"M={row['M']}", 
                (row['median_latency'], row['ndcg@10']),
                xytext=(8, 8), textcoords='offset points',
                fontsize=10, fontweight='bold', alpha=0.8)

# Add reference line for Flat index
ax.axhline(y=results_flat['ndcg@10'], color='red', linestyle='--', 
           linewidth=2, label=f"Flat Index (nDCG={results_flat['ndcg@10']:.4f})", alpha=0.7)

ax.set_xlabel('Median Latency (ms)', fontsize=12, fontweight='bold')
ax.set_ylabel('nDCG@10', fontsize=12, fontweight='bold')
ax.set_title('HNSW: Speed vs Quality Trade-off for Different M Values', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=11, loc='lower right')

plt.tight_layout()
plt.savefig('M_speed_vs_quality.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Plot saved as 'M_speed_vs_quality.png'")

In [ ]:
# Plot 2: M Parameter Multi-metric Analysis
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Plot 2a: Build Time vs M
axes[0, 0].plot(M_df['M'], M_df['build_time'], 
                marker='o', linewidth=2, color='red', markersize=8)
axes[0, 0].set_xlabel('M (Graph Connectivity)', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('Build Time (seconds)', fontsize=11, fontweight='bold')
axes[0, 0].set_title('Index Build Time vs M', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2b: Query Latency vs M
axes[0, 1].plot(M_df['M'], M_df['median_latency'], 
                marker='s', linewidth=2, color='steelblue', markersize=8)
axes[0, 1].fill_between(M_df['M'], 
                         M_df['median_latency'], 
                         M_df['p95_latency'], 
                         alpha=0.3, label='p50-p95 range')
axes[0, 1].set_xlabel('M (Graph Connectivity)', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('Latency (ms)', fontsize=11, fontweight='bold')
axes[0, 1].set_title('Query Latency vs M', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

# Plot 2c: nDCG vs M
axes[0, 2].plot(M_df['M'], M_df['ndcg@10'], 
                marker='^', linewidth=2, color='green', markersize=8)
axes[0, 2].axhline(y=results_flat['ndcg@10'], color='red', linestyle='--', 
                   linewidth=2, label='Flat Index', alpha=0.7)
axes[0, 2].set_xlabel('M (Graph Connectivity)', fontsize=11, fontweight='bold')
axes[0, 2].set_ylabel('nDCG@10', fontsize=11, fontweight='bold')
axes[0, 2].set_title('Retrieval Quality vs M', fontsize=12, fontweight='bold')
axes[0, 2].grid(True, alpha=0.3)
axes[0, 2].legend()

# Plot 2d: Recall vs M
axes[1, 0].plot(M_df['M'], M_df['recall@10'], 
                marker='D', linewidth=2, color='orange', markersize=8)
axes[1, 0].axhline(y=results_flat['recall@10'], color='red', linestyle='--', 
                   linewidth=2, label='Flat Index', alpha=0.7)
axes[1, 0].set_xlabel('M (Graph Connectivity)', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Recall@10', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Recall vs M', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()

# Plot 2e: QPS vs M
axes[1, 1].plot(M_df['M'], M_df['qps'], 
                marker='p', linewidth=2, color='purple', markersize=8)
axes[1, 1].set_xlabel('M (Graph Connectivity)', fontsize=11, fontweight='bold')
axes[1, 1].set_ylabel('Queries Per Second (QPS)', fontsize=11, fontweight='bold')
axes[1, 1].set_title('Throughput vs M', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

# Plot 2f: Build Time vs Quality (Trade-off)
axes[1, 2].scatter(M_df['build_time'], M_df['ndcg@10'], 
                   s=100, c=M_df['M'], cmap='viridis', alpha=0.7, edgecolors='black')
for idx, row in M_df.iterrows():
    axes[1, 2].annotate(f"M={row['M']}", 
                        (row['build_time'], row['ndcg@10']),
                        xytext=(5, 5), textcoords='offset points', fontsize=9)
axes[1, 2].set_xlabel('Build Time (seconds)', fontsize=11, fontweight='bold')
axes[1, 2].set_ylabel('nDCG@10', fontsize=11, fontweight='bold')
axes[1, 2].set_title('Build Cost vs Quality', fontsize=12, fontweight='bold')
axes[1, 2].grid(True, alpha=0.3)

fig.suptitle('HNSW Parameter Analysis: Impact of M (Graph Connectivity)', 
             fontsize=15, fontweight='bold', y=0.995)

plt.tight_layout()
plt.savefig('M_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Plot saved as 'M_analysis.png'")

In [ ]:
# Find the optimal M value (best overall balance)
# Calculate efficiency score: quality / (latency * build_time)
# This balances search quality, search speed, AND build cost
M_df['efficiency_score'] = M_df['ndcg@10'] / ((M_df['median_latency'] / 1000) * M_df['build_time'])

optimal_idx = M_df['efficiency_score'].idxmax()
optimal_M = M_df.loc[optimal_idx]

print("\n" + "="*90)
print("🎯 OPTIMAL M VALUE (Best Overall Balance)")
print("="*90)
print(f"\nBest efficiency score at M = {optimal_M['M']}")
print(f"\nMetrics:")
print(f"   • Build Time: {optimal_M['build_time']:.3f} seconds")
print(f"   • Median Latency: {optimal_M['median_latency']:.2f} ms")
print(f"   • P95 Latency: {optimal_M['p95_latency']:.2f} ms")
print(f"   • nDCG@10: {optimal_M['ndcg@10']:.4f}")
print(f"   • Recall@10: {optimal_M['recall@10']:.4f}")
print(f"   • QPS: {optimal_M['qps']:.1f} queries/second")
print(f"   • Efficiency Score: {optimal_M['efficiency_score']:.4f}")

print(f"\n📊 Comparison across all M values:")
print(f"\nLowest M ({M_df.iloc[0]['M']}):")
print(f"   • Build: {M_df.iloc[0]['build_time']:.3f}s (fastest build)")
print(f"   • Query: {M_df.iloc[0]['median_latency']:.2f}ms")
print(f"   • nDCG@10: {M_df.iloc[0]['ndcg@10']:.4f} ({(1-M_df.iloc[0]['ndcg@10']/results_flat['ndcg@10'])*100:.1f}% quality loss)")

print(f"\nBaseline M ({M_df[M_df['M']==16].iloc[0]['M']} - paper's choice):")
baseline_M = M_df[M_df['M']==16].iloc[0]
print(f"   • Build: {baseline_M['build_time']:.3f}s")
print(f"   • Query: {baseline_M['median_latency']:.2f}ms")
print(f"   • nDCG@10: {baseline_M['ndcg@10']:.4f} ({(1-baseline_M['ndcg@10']/results_flat['ndcg@10'])*100:.1f}% quality loss)")

print(f"\nHighest M ({M_df.iloc[-1]['M']}):")
print(f"   • Build: {M_df.iloc[-1]['build_time']:.3f}s (slowest build)")
print(f"   • Query: {M_df.iloc[-1]['median_latency']:.2f}ms")
print(f"   • nDCG@10: {M_df.iloc[-1]['ndcg@10']:.4f} ({(1-M_df.iloc[-1]['ndcg@10']/results_flat['ndcg@10'])*100:.1f}% quality loss)")

print(f"\nOptimal M ({optimal_M['M']}):")
print(f"   • Build: {optimal_M['build_time']:.3f}s")
print(f"   • Query: {optimal_M['median_latency']:.2f}ms")
print(f"   • nDCG@10: {optimal_M['ndcg@10']:.4f} ({(1-optimal_M['ndcg@10']/results_flat['ndcg@10'])*100:.1f}% quality loss)")

# Key insights
print(f"\n💡 KEY INSIGHTS:")

# Quality improvement from M=8 to M=16
quality_gain = (M_df[M_df['M']==16].iloc[0]['ndcg@10'] - M_df[M_df['M']==8].iloc[0]['ndcg@10']) / M_df[M_df['M']==8].iloc[0]['ndcg@10'] * 100
build_cost = (M_df[M_df['M']==16].iloc[0]['build_time'] - M_df[M_df['M']==8].iloc[0]['build_time']) / M_df[M_df['M']==8].iloc[0]['build_time'] * 100
print(f"   • Doubling M from 8→16 improves quality by {quality_gain:.1f}% but increases build time by {build_cost:.1f}%")

# Diminishing returns from M=16 to M=32
if len(M_df[M_df['M']==32]) > 0:
    quality_gain_high = (M_df[M_df['M']==32].iloc[0]['ndcg@10'] - M_df[M_df['M']==16].iloc[0]['ndcg@10']) / M_df[M_df['M']==16].iloc[0]['ndcg@10'] * 100
    build_cost_high = (M_df[M_df['M']==32].iloc[0]['build_time'] - M_df[M_df['M']==16].iloc[0]['build_time']) / M_df[M_df['M']==16].iloc[0]['build_time'] * 100
    print(f"   • Doubling M from 16→32 improves quality by only {quality_gain_high:.1f}% but increases build time by {build_cost_high:.1f}%")
    print(f"   • ⚠️  Diminishing returns: Higher M values are not cost-effective for this dataset")

print(f"\n🎯 RECOMMENDATION:")
print(f"   • For production: Use M={optimal_M['M']} (best balance of build cost, speed, and quality)")
print(f"   • Paper's choice (M=16) is {'optimal' if optimal_M['M'] == 16 else 'close to optimal'} for this dataset")
print(f"   • M < 16: Only if build time is critical and slight quality loss is acceptable")
print(f"   • M > 16: Only if you need maximum quality and can afford longer builds")
print("\n" + "="*90)

In [ ]:
# Save M tuning results
M_df.to_csv('M_tuning_results.csv', index=False)
print("✅ M parameter tuning results saved to 'M_tuning_results.csv'")

## 🎓 Final Comparison: efSearch vs M Parameter Impact

Let's compare the impact of both parameters side-by-side to understand which one matters more.

In [ ]:
# Compare parameter impacts
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left plot: Quality range for each parameter
axes[0].plot(efSearch_df['efSearch'], efSearch_df['ndcg@10'], 
             marker='o', linewidth=2, markersize=6, label='efSearch (runtime)', color='steelblue')
axes[0].set_xlabel('efSearch Value', fontsize=11, fontweight='bold')
axes[0].set_ylabel('nDCG@10', fontsize=11, fontweight='bold')
axes[0].set_title('efSearch Impact on Quality (No Rebuild Required)', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=results_flat['ndcg@10'], color='red', linestyle='--', linewidth=1.5, alpha=0.5, label='Flat Index')
axes[0].legend()

# Right plot: M parameter
axes[1].plot(M_df['M'], M_df['ndcg@10'], 
             marker='s', linewidth=2, markersize=8, label='M (build-time)', color='darkgreen')
axes[1].set_xlabel('M Value (Graph Connectivity)', fontsize=11, fontweight='bold')
axes[1].set_ylabel('nDCG@10', fontsize=11, fontweight='bold')
axes[1].set_title('M Impact on Quality (Rebuild Required)', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=results_flat['ndcg@10'], color='red', linestyle='--', linewidth=1.5, alpha=0.5, label='Flat Index')
axes[1].legend()

plt.tight_layout()
plt.savefig('parameter_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Plot saved as 'parameter_comparison.png'")

# Print summary statistics
print("\n" + "="*90)
print("📊 PARAMETER IMPACT SUMMARY")
print("="*90)

efSearch_quality_range = efSearch_df['ndcg@10'].max() - efSearch_df['ndcg@10'].min()
M_quality_range = M_df['ndcg@10'].max() - M_df['ndcg@10'].min()

print(f"\nefSearch Parameter (Runtime - No Rebuild):")
print(f"   • Quality range: {efSearch_quality_range:.4f} ({efSearch_quality_range/results_flat['ndcg@10']*100:.1f}% of Flat performance)")
print(f"   • Latency range: {efSearch_df['median_latency'].min():.2f}ms - {efSearch_df['median_latency'].max():.2f}ms")
print(f"   • Speedup range: {results_flat['median_latency']/efSearch_df['median_latency'].max():.1f}x - {results_flat['median_latency']/efSearch_df['median_latency'].min():.1f}x vs Flat")
print(f"   • Advantage: ✅ Fast to test, no rebuild needed")

print(f"\nM Parameter (Build-Time - Rebuild Required):")
print(f"   • Quality range: {M_quality_range:.4f} ({M_quality_range/results_flat['ndcg@10']*100:.1f}% of Flat performance)")
print(f"   • Build time range: {M_df['build_time'].min():.2f}s - {M_df['build_time'].max():.2f}s")
print(f"   • Latency range: {M_df['median_latency'].min():.2f}ms - {M_df['median_latency'].max():.2f}ms")
print(f"   • Advantage: ✅ Structural improvement, affects graph quality")

print(f"\n💡 KEY TAKEAWAYS:")
print(f"   • efSearch has {'LARGER' if efSearch_quality_range > M_quality_range else 'SMALLER'} impact on quality ({efSearch_quality_range:.4f} vs {M_quality_range:.4f})")
print(f"   • efSearch is easier to tune (no rebuild), M requires more computational cost")
print(f"   • Best strategy: Choose M based on quality needs, then tune efSearch for speed/quality balance")
print(f"   • For this dataset: M={optimal_M['M']}, efSearch={optimal_efSearch['efSearch']} is optimal")
print("="*90)